# Train

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/pafpn/faster_rcnn_r50_pafpn_1x_coco.py')

PREFIX = '../../input/data/'

epoch = 12

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_pafpn_1x_trash'


cfg.model.roi_head.bbox_head.num_classes = 11
print(cfg.model)

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

{'type': 'FasterRCNN', 'pretrained': 'torchvision://resnet50', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch'}, 'neck': {'type': 'PAFPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoIHead', 'bbox_roi_extractor': {'type': 'SingleRoIExtractor', 'roi_layer': {'type': 'RoIAlign', 'output_size': 7, 'sampling_ratio': 0}, 'out_channels

In [3]:
model = build_detector(cfg.model)

2021-05-11 04:20:03,998 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 04:20:03,999 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 04:20:04,251 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.54s)
creating index...
index created!


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-11 04:20:12,337 - mmdet - INFO - Start running, host: root@a1b075ba2232, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_r50_pafpn_1x_trash
2021-05-11 04:20:12,337 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs


Done (t=1.22s)
creating index...
index created!


2021-05-11 04:20:31,639 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 0:50:14, time: 0.386, data_time: 0.050, memory: 2159, loss_rpn_cls: 0.5449, loss_rpn_bbox: 0.0890, loss_cls: 0.6486, acc: 88.1113, loss_bbox: 0.1236, loss: 1.4060, grad_norm: 6.7127
2021-05-11 04:20:48,672 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 0:46:59, time: 0.341, data_time: 0.006, memory: 2159, loss_rpn_cls: 0.2337, loss_rpn_bbox: 0.0841, loss_cls: 0.4334, acc: 90.7246, loss_bbox: 0.3411, loss: 1.0922, grad_norm: 2.8973
2021-05-11 04:21:05,590 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-03, eta: 0:45:36, time: 0.338, data_time: 0.006, memory: 2159, loss_rpn_cls: 0.1748, loss_rpn_bbox: 0.0719, loss_cls: 0.4141, acc: 90.5479, loss_bbox: 0.3582, loss: 1.0190, grad_norm: 3.3244
2021-05-11 04:21:22,362 - mmdet - INFO - Epoch [1][200/655]	lr: 7.972e-03, eta: 0:44:41, time: 0.335, data_time: 0.006, memory: 2159, loss_rpn_cls: 0.1562, loss_rpn_bbox: 0.0793, loss_cls: 0.4330, acc: 89.2041, loss

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 21.8 task/s, elapsed: 30s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 21.8 task/s, elapsed: 30s, ETA:     0s

2021-05-11 05:11:43,250 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.64s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.01s).
Accumulating evaluation results...


2021-05-11 05:11:50,829 - mmdet - INFO - Epoch(val) [12][655]	bbox_mAP: 0.1580, bbox_mAP_50: 0.2690, bbox_mAP_75: 0.1570, bbox_mAP_s: 0.0430, bbox_mAP_m: 0.1420, bbox_mAP_l: 0.2950, bbox_mAP_copypaste: 0.158 0.269 0.157 0.043 0.142 0.295


DONE (t=0.84s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.158
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.269
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.157
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.043
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.142
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.295
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.315
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.108
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.281
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.567


# Test

In [11]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [12]:
cfg.model.train_cfg = None
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [13]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [14]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-11 05:24:47,970 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 05:24:47,971 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 05:24:48,229 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



Use load_from_local loader


In [15]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 21.6 task/s, elapsed: 39s, ETA:     0s

In [16]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.27755344 161.89021 73.92375 253.2754 163.2...,batch_01_vt/0021.jpg
1,1 0.14034286 286.60178 345.25772 308.23074 373...,batch_01_vt/0028.jpg
2,1 0.5135002 69.16179 165.50006 383.15405 381.7...,batch_01_vt/0031.jpg
3,2 0.5603375 298.53494 159.5227 350.06766 226.7...,batch_01_vt/0032.jpg
4,1 0.2884677 392.61008 267.3247 480.3569 339.09...,batch_01_vt/0070.jpg
